Now lets try this on text!
https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_20newsgroups.html#sphx-glr-auto-examples-linear-model-plot-sparse-logistic-regression-20newsgroups-py

In [48]:
from pathlib import Path
import os, sys
import numpy as np
from sklearn import datasets, svm, metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline


In [1]:
from sklearn.datasets import fetch_20newsgroups_vectorized

# Turn down for faster run time
n_samples = 5000

X, y = fetch_20newsgroups_vectorized(subset="all", return_X_y=True)
X = X[:n_samples]
y = y[:n_samples]



In [49]:
with open('clean_fake.txt', 'r') as f:
    fake = f.read().split('\n')

with open('clean_real.txt', 'r') as f:
    real = f.read().split('\n')

In [56]:
y = np.append(np.ones(np.shape(fake)), np.zeros(np.shape(real)))
X = fake + real

X_tf = count_vect.fit_transform(X)
tf_transformer = TfidfTransformer(use_idf=False).fit(X_tf)
X_tf = tf_transformer.transform(X_tf)

X_train, X_test, y_train, y_test = train_test_split(
    X_tf, y, random_state=42, stratify=y, test_size=0.3
)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tf, y, random_state=42, stratify=y, test_size=0.3
)

train_samples, n_features = X_train.shape
n_classes = np.unique(y).shape[0]

print(
    "Dataset 20newsgroup, train_samples=%i, n_features=%i, n_classes=%i"
    % (train_samples, n_features, n_classes)
)

nb = MultinomialNB(alpha=1)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

accuracy1 = np.sum(y_pred == y_test) / y_test.shape[0]

print ("accuracy= ", accuracy1)

print(
    f"Classification report for classifier {nb}:\n"
    f"{metrics.classification_report(y_test, y_pred)}\n"
)



Dataset 20newsgroup, train_samples=2287, n_features=5799, n_classes=2
accuracy=  0.8073394495412844
Classification report for classifier MultinomialNB(alpha=1):
              precision    recall  f1-score   support

         0.0       0.77      0.96      0.86       591
         1.0       0.91      0.57      0.70       390

    accuracy                           0.81       981
   macro avg       0.84      0.77      0.78       981
weighted avg       0.83      0.81      0.80       981




In [52]:
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning

mlp = MLPClassifier(
    hidden_layer_sizes=(65,),
    max_iter=100,
    alpha=1e-4,
    solver="sgd",
    random_state=1,
    learning_rate_init=0.2,
)
# fit
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
    mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

pred = mlp.predict(X_test)


Training set score: 1.000000
Test set score: 0.829766


In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=y, test_size=0.3
)

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('mlp', MultinomialNB()),
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.8226299694189603

In [87]:
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42,max_iter=5, tol=None)),
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.8267074413863404

In [9]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_train[:40], y_train[:40])

In [40]:
from sklearn.model_selection import GridSearchCV

X, y = fetch_20newsgroups(subset="all", return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=y, test_size=0.7
)
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (0.002, 0.003),
}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)


In [41]:
predicted = gs_clf.predict(X_test)
np.mean(predicted == y_test)

0.8796331387857197